In [13]:
from numba import cuda  
import numpy as np
import numba

In [ ]:
def intg_item(N, p, T, S0, r, l, g, sigma) -> list:
    """
    模拟股票的随机过程
    N : 分区数量
    T : 总时间
    h : 时间步长

    S0 : 初始值
    r : 
    l :
    g : 
    sigma :
    """
    dt = T/(N*p)
    t = np.linspace(0, T/p, N)
    W = np.zeros(N)
    lgSt = np.zeros(N)
    W[1:] = np.cumsum(np.sqrt(dt)*np.random.standard_normal(size=N-1))  # 累积和
    lgSt[0] = np.log(S0)
    lgSt[1:] = np.cumsum((r - l - 0.5 * sigma**2) * dt + sigma * (W[1:] - W[0:-1]))  # 迭代生成
    St = S0 * np.exp(lgSt)
    # print("st",St)

    final_int = np.zeros(N)
    for i in range(N):
        final_int[i] = np.exp(-r * t[i]) * np.maximum(np.exp(g*t[i]) - St[i], 0)
        
        # final_int[i] = np.exp(-r * t[i]) * np.maximum(np.exp(g*t[i]), St[i])
        # print("exp(-r * t[i])",np.exp(-r * t[i]))
        # print("maximum",np.maximum(np.exp(g*t[i]) - St[i], 0))
        # print("final_int[i]",final_int[i])

    return t, final_int

In [ ]:
@cuda.jit
def single_mc(d_int_items, d_texp_all, d_results, d_fee_results, d_buff_day, k, l, T, d_ltp):
    idx = cuda.grid(1)
    for w in range(T):
        # print(int((w + 1) * d_buff_day) - 1)
        d_results[idx] += d_int_items[idx][int((w + 1) * d_buff_day) - 1] * d_np_interval_P_cache[k][w+k]
        d_fee_results[idx] += d_texp_all[int((w+1)*d_buff_day) - 1]*d_np_interval_P_cache[k][w+k]
        # print("int_death_payoff", int_death_payoff)
    d_results[idx] = d_results[idx] + d_int_items[idx][-1] * d_terminal_live_cache[k]
    d_fee_results[idx] = 1 - d_fee_results[idx] - d_terminal_live_cache[k]*d_ltp

    # print("fee",fee)

    # result.append(intern_res)
    # ave_j = np.mean(result)
    # ave.append(ave_j)###用于画图，看结果稳不稳定
    # fee_result.append(fee)


def MC(M, k, p, buff_day, S0, r, g, x0, T0, lifespan, l):
    T = (lifespan - x0) * p - k
    # print('T', T)
    N = buff_day * T
    # print('N', N)
    sigma = 0.2

    # 你现有的MC函数代码继续
    result = []
    avg_res = []
    ave = []
    fee_result = []

    M = 1000000
    d_results = cuda.to_device(np.zeros(M, dtype=np.float32))
    d_fee_results = cuda.to_device(np.zeros(M, dtype=np.float32))

    # 定义线程和块的数量
    threads_per_block = 256
    blocks_per_grid = (M + threads_per_block - 1) // threads_per_block

    int_items = []
    
    for i in range(M):
        t_all, int_item = ST.intg_item(N, p, T, S0, r, l, g, sigma)
        int_items.append(int_item)


    d_init_items = cuda.to_device(np.stack(tuple(int_items), axis=0))
    d_texp_all = cuda.to_device(np.exp(-l*t_all))
    d_buffer_day = cuda.to_device(buff_day)
    d_ltp = cuda.to_device(np.exp(-l*T/p))
    
    # 执行CUDA核函数
    single_mc[blocks_per_grid, threads_per_block](d_init_items, d_texp_all, d_results, d_fee_results, d_buffer_day, k, l, T, d_ltp)

    # 将结果传回CPU
    results = d_results.copy_to_host()
    fees = d_fee_results.copy_to_host()
    avg_res = np.mean(results)
    fee_avg_res = np.mean(fees) 

    return avg_res, fee_avg_res, ave